# 6. 分散分析（１要因）

## 6.1　なぜt検定が使えないか

- URL : http://kogolab.chillout.jp/elearn/hamburger/chap6/sec1.html
- テーマ
   1. 読み物オンリーページ
       - t検定 = 2つの標本間の平均の差を調べる
       - 3つ以上の標本間の差を調べるのに向かない事のイメージ


## 6.2　分散分析とは

- URL : http://kogolab.chillout.jp/elearn/hamburger/chap6/sec2.html
- テーマ
   1. これまでの復習
    - csvファイルの読み込み
    - 統計量の算出(平均値、標本標準偏差)
   2. 複数データの取り扱い
    - これまでは基本的に2つの標本データを扱ってきたが、ここでは3つ以上の列データを取り扱う
    - 3つのデータを連結したものも使用する

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import math

# csvからデータの読み込み
df_1 = pd.read_csv('./data/chapter6-2.csv')

# 3列分のデータを全てまとめたデータを作成(ついでに平均値と標準偏差も算出)
# そのまま3つのデータを連結すると行番号が重複するので、行番号は振りなおす(ignore_index = True)
# データの連結が列名を使った直接指定・・・何か良い方法は無いか？
sum_1    = pd.concat([df_1.wakwak, df_1.mogmog, df_1.pakpak], ignore_index=True)
sum_1_df = pd.DataFrame({'sum' : [sum_1.mean(), sum_1.var(ddof=0), sum_1.std(ddof=0)]},
                      index=['mean', 'var', 'std'])

# 算出した統計量を表にまとめる
mean_var_list = pd.concat([df_1.mean(), df_1.var(ddof=0), df_1.std(ddof=0)], axis=1).T
mean_var_list.index = ['mean', 'var', 'std']
mean_var_list

# 表示
df_1_stat = pd.concat([mean_var_list, sum_1_df], axis=1)
df_1_stat

,wakwak,mogmog,pakpak,sum
mean,84.000000,79.500000,88.150000,83.883333
var,29.000000,29.750000,28.527500,41.569722
std,5.385165,5.454356,5.341114,6.447459


## 6.3　分散分析をしよう

- URL : http://kogolab.chillout.jp/elearn/hamburger/chap6/sec3.html
- テーマ
   1. 分散分析の実施
    - 今回は一元配置分散分析に相当？
    - 群内のズレと群間のズレに注目する
   2. 関数の使用と地道な計算
    - stats.f_onewayを使った場合と、地道に順を追って計算する場合の2通りで確認する
        - 2つの計算結果が近い値になることを確認する
   3. pythonの基本構文の使用
    - for文、配列への追加(append)
- その他
    - 分散分析の参考URL
        - http://www.shiga-med.ac.jp/~koyama/stat/com-anova.html
        - https://mkacky.wordpress.com/2013/05/22/numpy%E3%81%A7%E5%88%86%E6%95%A3%E5%88%86%E6%9E%90/

In [2]:
# scipyの関数を使用した場合
f_oneway_list_1 = stats.f_oneway(df_1.wakwak, df_1.mogmog, df_1.pakpak)
print('f_val = ', f_oneway_list_1[0])
print('p_val = ', f_oneway_list_1[1])

f_val =  12.2231101945
p_val =  3.82482645839e-05


In [3]:
# 手計算に近い順序で確認実施
# 1) 郡内の平方和算出
sum_sq_in_1 = []
for i in range(0, len(df_1.columns)) :
    sum_sq_in_1.append(df_1.ix[:,i].var(ddof=0) * len(df_1.ix[:,i]))

# 2) 群間の平方和算出
sum_sq_bt_1 = []
for i in range(0, len(df_1.columns)) :
    sum_sq_bt_1.append((df_1.ix[:,i].mean() - sum_1.mean())**2 * len(df_1.ix[:,i]))

# 3) 全体の平方和算出
sum_sq_in_sum_1    = sum(sum_sq_in_1)
sum_sq_bt_sum_1    = sum(sum_sq_bt_1)

# 4) 自由度算出
ddof_in  = (len(df_1) - 1) * len(df_1.columns)
ddof_bt  = len(df_1.columns) - 1

# 5) 平均平方算出
mean_sq_in_1 = sum_sq_in_sum_1 / ddof_in
mean_sq_bt_1 = sum_sq_bt_sum_1 / ddof_bt

# 6) F値算出
f_val_1 = mean_sq_bt_1 / mean_sq_in_1

# 表示
print('f_val(library)     = ', f_oneway_list_1[0])
print('f_val(not library) = ', f_val_1)

f_val(library)     =  12.2231101945
f_val(not library) =  12.223110194494591


## 6.4　実践編：3種類のハンバーガーの比較

- URL : http://kogolab.chillout.jp/elearn/hamburger/chap6/sec4.html
- テーマ
   1. 分散分析の実施
    - csvファイルからデータを読み込み → 関数を使用してF値を求める

In [4]:
# csvからデータの読み込み
df_2 = pd.read_csv('./data/chapter6-4.csv')

# 関数を使用してF値/p値を算出
f_oneway_list_2 = stats.f_oneway(df_2.wakwak, df_2.mogmog, df_2.pakpak)
print('f_val = ', f_oneway_list_2[0])
print('p_val = ', f_oneway_list_2[1])

f_val =  4.58047493404
p_val =  0.0158680849582


## 6.4　実践編：3種類のハンバーガーの比較

- URL : http://kogolab.chillout.jp/elearn/hamburger/chap6/sec4.html
- テーマ
   1. 分散分析の実施
    - csvファイルからデータを読み込み → 関数を使用してF値を求める

In [5]:
# csvからデータの読み込み
df_3 = pd.read_csv('./data/chapter6-9.csv')

# 関数を使用してF値/p値を算出 : 3つのデータの個数をそれぞれバラバラなので、dropna()を使用してNanを削除
f_oneway_list_3 = stats.f_oneway(df_3.no_manga.dropna(), df_3.old_manga.dropna(), df_3.new_manga.dropna())
print('f_val = ', f_oneway_list_3[0])
print('p_val = ', f_oneway_list_3[1])

f_val =  5.78724488428
p_val =  0.00602720804494
